Preparación de datos y análisis para fine-tuning de chat model


In [2]:
!pip install tiktoken

In [3]:
import json
import tiktoken
import numpy as np
from collections import defaultdict

Cargar dataset desde archivo JSONL


In [5]:
data_path = 'C:\\Users\\Guido\\Desktop\\Cursos\\Platzi\\Desarrollo de Chatbots con OpenAI\\FineTuning\\RickAndMorty\\data_RickAndMorty.jsonl'

In [6]:
#Cargar dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

#Análisis inicial de dataset
print("Num examples: ", len(dataset))
print("First example: ")
for message in dataset[0]["messages"]:
    print(message)   

Num examples:  19
First example: 
{'role': 'system', 'content': 'Eres un asistente experto en la serie Rick and Morty, capaz de responder preguntas sobre personajes, episodios, teorías y curiosidades del show.'}
{'role': 'user', 'content': '¿Quién es Rick Sánchez?'}
{'role': 'assistant', 'content': 'Rick Sánchez (Rick C-137) es un científico genial pero problemático, considerado el hombre más inteligente del universo. Es el abuelo de Morty y viaja a través de dimensiones y galaxias con su nieto. Es conocido por su personalidad cínica, su alcoholismo y su frase característica: "Wubba Lubba Dub Dub!"'}


Validación de formato

In [7]:
# Revisión de errores de formato
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found") 

No errors found


Utilidades de conteo de Tokens


In [8]:
encoding = tiktoken.get_encoding("cl100k_base")

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

Conteo de Tokens y advertencias de Data.


In [ ]:
# Advertencias y conteo de tokens
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Número de ejemplos con mensaje de sistema faltante:", n_missing_system)
print("Número de ejemplos con mensaje de usuario faltante:", n_missing_user)
print_distribution(n_messages, "núm_mensajes_por_ejemplo")
print_distribution(convo_lens, "núm_total_tokens_por_ejemplo")
print_distribution(assistant_message_lens, "núm_assistant_tokens_por_ejemplo")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} Si los ejemplos superan el límite de 4096 tokens, se truncarán durante el ajuste.")